In [7]:
import numpy as np


class Gate:
    def forward(self):
        raise NotImplementedError

    def backward(self):
        raise NotImplementedError


# Example of an AddGate class inheriting from the Gate class 
import numpy as np
class AddGate(Gate): 
    def forward(self, x, y): 
        self.x = x 
        self.y = y 
        return x + y 
    def backward(self, dz): 
        dx = dz * np.ones_like(self.x) 
        dy = dz * np.ones_like(self.y) 
        return dx, dy 
    # Example of a MultiplyGate class inheriting from the Gate class
import numpy as np

class MultiplyGate:
    def forward(self, x, y):
        self.x = x
        self.y = y
        return np.dot(x, y)

    def backward(self, dz):
        # print("multipllly")
        # print(dz.shape)
        # print(self.y.shape)
        # print(self.x.shape)
        dz=dz.reshape(self.x.shape[0],1)
        self.y=self.y.reshape(self.y.shape[0],1)
        
        dx = np.dot(dz, self.y.T)
        dy = np.dot(self.x.T, dz)
        # print(dx.shape)
        # print(dy.shape)
        return dx, dy

In [8]:
# Example of a Softmax activation function
class SoftmaxActivation(Gate):
    def forward(self, x):
        self.x = x
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    def backward(self, dz):
        softmax_x = self.forward(self.x)
        dx = dz * softmax_x * (1 - softmax_x)
        return dx
# Example of a Sigmoid activation function
class SigmoidActivation(Gate):
    def forward(self, x):
        self.x = x
        return 1 / (1 + np.exp(-x))

    def backward(self, dz):
        sigmoid_x = 1 / (1 + np.exp(-self.x))
        # print("in Sigmoid activation")
        # print(dz.shape)
        # print(self.x.shape)
        # print(sigmoid_x.shape)
        dx = dz * sigmoid_x * (1 - sigmoid_x)
        
        print(dx.shape)

        return dx
# Example of a ReLU activation function
class ReLUActivation(Gate):
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)
    

    def backward(self, dz):
        dx = dz * np.where(self.x > 0, 1, 0)
        return dx

In [9]:
# Example of Binary Cross-Entropy (BCE) loss function
class BinaryCrossEntropyLoss(Gate):
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def backward(self,y_pred, y_true):
        dx = (y_pred - y_true) / (y_pred * (1 - y_pred))
        return dx

In [10]:
# Example of L2 loss function
class L2Loss(Gate):
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return 0.5 * np.mean((y_pred - y_true) ** 2)

    def backward(self,y_pred, y_true):
        dx = y_pred - y_true
        return dx

In [11]:
class Model:
    def __init__(self,layers_dim, actiation_func, loss):
        self.layers=[]
        self.activFunc=[]
        self.layers_dim = layers_dim
        self.num_layers=len(layers_dim)
        self.grads=[]
        self.weightsgrads=[]
        self.biasgrads=[]
        if len(layers_dim)!=len(actiation_func)+1:
            raise ValueError("the number of layers is not equal to the number of activation funcs")
        for i in range(0,len(layers_dim)-1):
            layer=[]
            layer.append(MultiplyGate())
            layer.append(AddGate())
            self.layers.append(layer)
        for i in range(0,len(actiation_func)):
            if actiation_func[i] =='sigmoid':
                self.activFunc.append(SigmoidActivation())
            
            if actiation_func[i] =='relu':
                self.activFunc.append(ReLUActivation())
            
            if actiation_func[i] =='softmax':
                self.activFunc.append(SoftmaxActivation())
            # print(type(self.activFunc[i]))
        if loss=='CE':
            self.loss=BinaryCrossEntropyLoss()
        elif loss=='L2':
            self.loss=L2Loss()
        self.parameters=self.initialise_parameters()
        # print(len(self.parameters))




    def initialise_parameters(self):
        parameters = []
        for i in range(1, len(self.layers_dim)):
            weights = np.random.randint(low=0,high=3, size=(self.layers_dim[i],self.layers_dim[i-1])) 
            biases = np.random.randint(low=0,high=3, size=(self.layers_dim[i])) 
            layer_param = {'weights': weights, 'biases': biases}
            parameters.append(layer_param)
            # print((parameters[i-1]['weights'].shape))
            # print((parameters[i-1]['biases'].shape))
        return parameters
    



    
    def forward(self, x):
        for i in range(0,self.num_layers-1):
            x = self.layers[i][0].forward( self.parameters[i]['weights'],x)
            # print(x.shape)
            x=x.flatten()
            x = self.layers[i][1].forward(x, self.parameters[i]['biases'])
            # print(x.shape)
            # print("actx,bef",x.shape)
            x = self.activFunc[i].forward(x)

            # print("actx,",x)

        return(x)
        


    def backward(self,losscomputed):
        dy=losscomputed
        self.grads.append(dy)
        dz=dy
        # print(dy.shape)
        for i in range(self.num_layers-1,0,-1):
            print("iter",i)
            dz=self.activFunc[i-1].backward(dz)
            # print("aft",dz.shape)
            dz=dz.flatten()
            # print(dz.shape)
            if len(dz)==1:
                dz=dz[0]
            dw,dz=self.layers[i-1][0].backward(dz)
            db=self.layers[i-1][1].backward(dz)
            self.weightsgrads.append(dw)
            # print(dz.shape)
            dz=dz.flatten()
            self.biasgrads.append(db)
            # print(len(self.weightsgrads))
            # print(len(self.biasgrads))
        
            

    def train(self, X_train, Y_train, learning_rate, num_epochs, gradient_descent_method='batch',batch_size=None,patience=10):
        for i in range (0,num_epochs):
            
            
            if gradient_descent_method=='batch':
                y_pred=self.predict(X_train)
                loss=self.loss.forward(y_pred,Y_train)
                print(loss)
                computed_loss=self.loss.backward(loss,Y_train)
                computed_loss=np.array(computed_loss)
                computed_loss=np.mean(computed_loss)
                print(computed_loss)
                self.backward(computed_loss)
                self.update_parameters(learning_rate)

                

            

    def update_parameters(self,learning_rate):
        for i in range(0,len(self.parameters)):
            self.parameters[i]['weights']=self.parameters[i]['weights']-learning_rate*self.weightsgrads[i]
            bias_grads_array = np.array(self.biasgrads[i])
            # Perform the multiplication with the learning rate
            self.parameters[i]['biases'] = self.parameters[i]['biases'] - learning_rate * bias_grads_array
        pass
    def predict(self, X):
        y_pred=[]
        for i in range(0,len(X)):
            y=self.forward(X[i])
            y_pred.append(y)
            # print(y)
        y_pred=np.array(y_pred)
        print(y_pred.shape)
        return (y_pred)
    
        



            


        
        

In [12]:
import pandas as pd
dataset=pd.read_csv("./training_data.csv")
y=dataset.iloc[0:100:,4:5].values

In [13]:

X=dataset.iloc[::,3:4].values

print((len(X[0])))


1


In [14]:
dim=[len(X[0]),8,3,1]
fun=['relu','relu','sigmoid']

k=Model(dim,fun,'L2')
y=np.random.randint(low=0 ,high=1,size=(569))
print(y.shape)
k.train(X,y,num_epochs=20,learning_rate=0.1)


(569,)
(111479, 1)
0.5
0.5
iter 3
(1,)
iter 2
iter 1


ValueError: shapes (8,3) and (1,) not aligned: 3 (dim 1) != 1 (dim 0)